# Tarea 5: Challenger experiments

Samantha Sánchez Tinoco

In [1]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/samantha.sancheztin@gmail.com/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

## 1. Lectura de datos

In [2]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [4]:
df_train = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-02.parquet')

## 2. Pre-procesamiento

In [5]:
# Feature engineering
def create_features(df):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    return df[categorical + numerical].to_dict(orient='records')

In [6]:
def fit_transform_dict_vectorizer(train_dicts):
    dv = DictVectorizer()
    return dv, dv.fit_transform(train_dicts)

def transform_dict_vectorizer(df, dv):
    test_dicts = create_features(df)
    return dv.transform(test_dicts)

In [7]:
# Preprocesamiento sin data leakage
train_dicts = create_features(df_train)
dv, X_train = fit_transform_dict_vectorizer(train_dicts)

# Transformar datos de validación
X_val = transform_dict_vectorizer(df_val, dv)

In [8]:
# Extraer variable objetivo
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
# Crear datasets MLflow
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

## 3. Modelos

### Gradient Boost